In [1]:
#pip install tradingview-ta
#pip install yfinance[optional]
#pip install investpy

from tvDatafeed import TvDatafeed, Interval
#import investpy
#import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

tv = TvDatafeed()

you are using nologin method, data you access may be limited


In [2]:
main_index = ["PX","PSECZ"]

tickers = ["BUX",'BET','BTE','CRE','WIG','AEX','CAC','DAX','FTMIB','UKX','IBC','ATX','OMXS30','OMXN40','SX5E','SMI',
           'EURCZK','EURHUF','EURRON','EURPLN','EURGBP']        

exchanges = ["BET",'BVB','VIE','VIE','GPW','EURONEXT','EURONEXT','XETR','TVC','FTSE','BME','VIE','OMXSTO',
             'OMXNORDIC','TVC','SIX',
             'OANDA','OANDA','SAXO','OANDA','OANDA']
             
def get_data(main_index, tickers, exchanges):
    try:
        data = pd.DataFrame(tv.get_hist(symbol=main_index[0],exchange=main_index[1],interval=Interval.in_daily,n_bars=7255))
            .rename(columns={"close": main_index[0]})
        data = data.drop(data.columns[5], axis=1)
        data.index = data.index.date
        
        for ticker, exchange in zip(tickers, exchanges):
            df = tv.get_hist(ticker, exchange, interval=Interval.in_daily, n_bars=7500)
            df = pd.DataFrame(df['close']).rename(columns={'close': ticker})
            df.index = df.index.date  # Změna datetime indexu na date index
            data = pd.merge(data, df, how='left', left_index=True, right_index=True)

        data = data.reset_index()
        data.rename(columns={'index': 'date'}, inplace=True) 
        data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
    
        return data
    except:
        print(ticker, exchange)

df = get_data(main_index, tickers, exchanges)
df

,date,symbol,open,high,low,PX,BUX,BET,BTE,CRE,...,ATX,OMXS30,OMXN40,SX5E,SMI,EURCZK,EURHUF,EURRON,EURPLN,EURGBP
0,1995-01-19,PSECZ:PX,552.20001,552.20001,552.20001,552.20001,NaN,NaN,NaN,NaN,...,NaN,291.429993,NaN,1319.01,2595.5000,NaN,NaN,NaN,NaN,NaN
1,1995-01-20,PSECZ:PX,547.90002,547.90002,547.90002,547.90002,NaN,NaN,NaN,NaN,...,NaN,290.070007,NaN,1305.78,2574.6001,NaN,NaN,NaN,NaN,NaN
2,1995-01-23,PSECZ:PX,543.50000,543.50000,543.50000,543.50000,NaN,NaN,NaN,NaN,...,NaN,285.720001,NaN,1288.91,2525.3000,NaN,NaN,NaN,NaN,NaN
3,1995-01-24,PSECZ:PX,539.59998,539.59998,539.59998,539.59998,NaN,NaN,NaN,NaN,...,NaN,289.369995,NaN,1292.01,2534.8000,NaN,NaN,NaN,NaN,NaN
4,1995-01-25,PSECZ:PX,535.59998,535.59998,535.59998,535.59998,NaN,NaN,NaN,NaN,...,NaN,288.000000,NaN,1294.99,2525.5000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7250,2024-01-11,PSECZ:PX,1451.25000,1463.00000,1451.25000,1456.26000,63066.570,15732.52,2237.3000,1729.08,...,3398.2300,2336.245600,2536.5168,4442.27,11153.6200,24.63855,378.9300,4.972610,4.35790,0.858660
7251,2024-01-12,PSECZ:PX,1456.10000,1463.12000,1454.61000,1457.47000,63628.129,15776.45,2266.4199,1730.70,...,3398.6101,2355.964600,2551.2173,4480.01,11226.4000,NaN,NaN,4.973005,NaN,NaN
7252,2024-01-15,PSECZ:PX,1456.22000,1460.64000,1454.22000,1457.28000,63787.070,15803.32,2253.9600,1729.90,...,3410.3501,2340.467300,2539.8203,4454.69,11207.5100,24.74405,379.4470,4.973670,4.39185,0.860760
7253,2024-01-16,PSECZ:PX,1457.89000,1458.54000,1445.81990,1445.83000,63292.898,15780.71,2255.6299,1736.01,...,3383.8301,2323.784900,2527.6460,4446.52,11229.6500,24.78190,380.8775,4.976470,4.39920,0.857855


### Dataset cleaning

In [44]:
def cleaning(df, start_date, end_date):

    # Checsk if start date is before the first date of the main index or end date is after the last date of the main index
    if start_date < df.date.min() or end_date > df.date.max() or start_date > end_date:
        print("start date is before the first date of the main index or end date is after the last date of the main index")
    else: 
        start_date = pd.to_datetime(start_date).strftime('%Y-%m-%d')
        end_date = pd.to_datetime(end_date).strftime('%Y-%m-%d')

    # Create DataFrame for given start_date and end_date as set in the function   
        filter_df = df[df["date"] >= start_date]
        filter_df = filter_df[filter_df["date"] <= end_date]
    # Count number of colums having minimal one NaN value
        nan_counts = filter_df.isna().sum() 
        rows = filter_df.shape[0]
    # Identify columns that have more than 1/3 of NaN values - these will be removed
        col_to_delete = nan_counts[nan_counts > rows/3].index
    # Drop columns that were identified to be removed
        clean_df = filter_df.drop(col_to_delete, axis=1) 
    # Drop NaN rows
        clean_df = clean_df.dropna()
    # Reset index
    clean_df = clean_df.reset_index(drop=True)

    # Confirm, if start_date was moved
    if start_date < clean_df['date'].iloc[0]:
        start_date_diff = pd.to_datetime(clean_df['date'].iloc[0]) - pd.to_datetime(start_date)
        cleaned_start_date = f'Start date was moved by {start_date_diff} on: {clean_df["date"].iloc[0]}'
    else:
        cleaned_start_date = f'No change in start date: {start_date}'
        
    # Confirm, if end_date was moved
    if end_date < clean_df['date'].iloc[-1]:
        end_date_diff = pd.to_datetime(clean_df['date'].iloc[-1]) - pd.to_datetime(end_date)
        cleaned_end_date = f'End date was moved by {end_date_diff} on: {clean_df["date"].iloc[-1]}'
    else:
        cleaned_end_date = f'No change in end date: {end_date}'

    # Convert all 'clean_df' dataset into EUR
    clean_df.iloc[:,2:6] = clean_df.iloc[:,2:6].div(clean_df['EURCZK'], axis=0)
    clean_df['BUX'] = clean_df['BUX'] / clean_df['EURHUF']
    clean_df['BET'] = clean_df['BET'] / clean_df['EURRON']
    clean_df['WIG'] = clean_df['WIG'] / clean_df['EURPLN']
    clean_df['UKX'] = clean_df['UKX'] / clean_df['EURGBP']

    clean_df.to_csv(r"clean_df.csv", index=False)

    return col_to_delete, cleaned_start_date, cleaned_end_date, clean_df

col_to_delete, cleaned_start_date, cleaned_end_date, clean_df = cleaning(df, '2015-01-01', df.date.max()) # start_date je konec ekonomicke krize, dle Index prosperity Deloitte to bylo v 4ervnu 2015
#col_to_delete, cleaned_start_date, cleaned_end_date, clean_df = cleaning(df, '2016-07-01', df.date.max())
#clean_df, col_to_delete, cleaned_start_date, cleaned_end_date = cleaning(df, '2007-07-01', '2018-07-01')

print(f'Deleted columns: {col_to_delete}')
print(f'Columns in the cleaned DataFrame: {clean_df.columns}')
print("-------------")
print(cleaned_start_date)
print(cleaned_end_date)
print("-------------")
print(clean_df)


Deleted columns: Index([], dtype='object')
Columns in the cleaned DataFrame: Index(['date', 'symbol', 'open', 'high', 'low', 'PX', 'BUX', 'BET', 'BTE',
       'CRE', 'WIG', 'AEX', 'CAC', 'DAX', 'FTMIB', 'UKX', 'IBC', 'ATX',
       'OMXS30', 'OMXN40', 'SX5E', 'SMI', 'EURCZK', 'EURHUF', 'EURRON',
       'EURPLN', 'EURGBP'],
      dtype='object')
-------------
Start date was moved by 4 days 00:00:00 on: 2015-01-05
No change in end date: 2024-01-15
-------------
            date    symbol       open       high        low         PX  \
0     2015-01-05  PSECZ:PX  34.782877  34.850841  34.041415  34.076119   
1     2015-01-07  PSECZ:PX  33.706575  33.722356  33.453722  33.474166   
2     2015-01-08  PSECZ:PX  33.177158  33.709234  33.176446  33.646324   
3     2015-01-12  PSECZ:PX  33.666432  33.846485  33.583167  33.678531   
4     2015-01-13  PSECZ:PX  33.757185  34.351701  33.584573  34.351701   
...          ...       ...        ...        ...        ...        ...   
1582  2024-01-08  P